In [1]:
# load environment variables from .env file
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
# tested agent creation and invocation with gpt-5-nano
from langchain.agents import create_agent
from langchain.messages import HumanMessage

agent = create_agent(model="gpt-5-nano")
question = HumanMessage(content="What is the capital of India?")

response = agent.invoke({"messages": [question]}) 
print(response['messages'][1].content)

New Delhi. It is the capital city of India and houses the central government’s key institutions.


In [4]:
# Lets create a tool for agent to use
from langchain.tools import tool
from tavily import TavilyClient

@tool
def web_search(query: str) -> str:
    "search the web for the query and return the results"
    return TavilyClient().search(query)

In [5]:
system_prompt = """

You are a personal chef. The user will give you an image of items in the fridge in their house.

Using the web search tool, search the web for recipes that can be made with the ingredients that are in the image.

Return recipe suggestions and eventually the recipe instructions to the user, if requested.

"""

In [6]:
# upload image
from ipywidgets import FileUpload
from IPython.display import display

uploader = FileUpload(accept='.png', multiple=False)
display(uploader)

FileUpload(value=(), accept='.png', description='Upload')

In [8]:
import base64

# Get the first (and only) uploaded file dict
uploaded_file = uploader.value[0]

# This is a memoryview
content_mv = uploaded_file["content"]

# Convert memoryview -> bytes
img_bytes = bytes(content_mv)  # or content_mv.tobytes()

# Now base64 encode
img_b64 = base64.b64encode(img_bytes).decode("utf-8")

In [9]:
# chef agent with system prompt and tool
chef_agent = create_agent(
    model="gpt-5-nano",
    system_prompt=system_prompt,
    tools=[web_search]
)

In [10]:
multimodal_question = HumanMessage(content=[
    {"type": "text", "text": "Tell me what to cook with the items in this image:"},
    {"type": "image", "base64": img_b64, "mime_type": "image/png"}
])

response = chef_agent.invoke(
    {"messages": [multimodal_question]}
)

print(response['messages'][-1].content)

Nice! You’ve got peppers (red, yellow, green), carrots, tomatoes, cucumber, avocado, greens, eggs, cheese, milk, some sausage/meat, and milk/dairy. Here are some quick ideas that use what you have. Pick a vibe and I’ll give you full steps or a full recipe.

- Cheesy veggie frittata (peppers, onions if you have them, eggs, cheese)
  - Why it’s good: fills you up with protein and veggies; you can fold in greens too.
  - How it uses your items: eggs + cheese + bell peppers + onion (if you have) + greens.
  - Quick version: sauté chopped peppers (and onions) until tender, whisk eggs with a splash of milk, pour over veggies, sprinkle cheese, cook on stove then finish under the broiler or in a hot oven until set (about 20–25 minutes total).

- Sausage, peppers, and onions skillet
  - Why it’s good: one-pan dinner, lots of flavor from peppers.
  - How it uses your items: sausage + bell peppers + onion (optional); you can add sliced carrot shreds for sweetness.
  - Quick version: brown sausage